In [1]:
import cv2 
import numpy as np
net = cv2.dnn.readNet('/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom_last.weights copy 5','/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom.cfg')
with open("/Users/amariofausta/Documents/Kuliah/Semester 8/Dataset/dataset/classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [2]:
import time
import sys

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
measurement_data = []
jumlah_frame = 0
detik = 0
Pembatas = []
Pembatas_bawah = 0
frame_count = 0
catatan_waktu1 = []
catatan_waktu2 = []
start_time_keseluruhan = time.time()

# Start video capture from webcam
cap = cv2.VideoCapture('/Users/amariofausta/Documents/Kuliah/Semester 8/Video mentah/30 cm part 2.mp4')
while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    
    # Create a blob from the input image
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    
    # Pass the blob through the network and get the outputs
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    # Process the outputs to get the detected objects
    class_ids = []
    confidences = []
    boxes = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.4:
                # Object detected
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, width, height])
        
    # Apply non-maximum suppression to remove redundant detections
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    # Draw the detected objects on the image
    for i in indices:
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        measurement_data.append([y, classes[class_ids[i]]])
        cv2.putText(frame, classes[class_ids[i]], (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        #print(boxes[i],classes[class_ids[i]])

        #Mengetahui pembatas atas dan bawah
        if classes[class_ids[i]] == 'Pembatas':
            Pembatas.append(boxes[i])
        if len(Pembatas) == 2:
            if Pembatas[0][1] > Pembatas[1][1]:
                Pembatas_atas = Pembatas[1][1]
                Pembatas_bawah = Pembatas[0][1]
            elif Pembatas[0][1] < Pembatas[1][1]:
                Pembatas_atas = Pembatas[0][1]
                Pembatas_bawah = Pembatas[1][1]

        if classes[class_ids[i]] == 'Bola':
            if len(Pembatas) > 2:
                if boxes[i][1] >= Pembatas_atas and boxes[i][1] <= Pembatas_bawah:
                    catatan_waktu1.append(time.time())
                if boxes[i][1] >= Pembatas_bawah:
                    catatan_waktu2.append(time.time())

    # Display the output image
    cv2.imshow("Output", frame)
    frame_count += 1
    jumlah_frame += 1
    
    end_time_keseluruhan = time.time()
    elapsed_time = end_time_keseluruhan - start_time_keseluruhan
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        frame_count = 0
        start_time_keseluruhan = time.time()
        print(fps)
    # Wait for a key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



# Release resources
cap.release()
cv2.destroyAllWindows()


1.2536935921740995


1   HIToolbox                           0x00000001b7ddc5c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x00000001b7ddc188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x00000001b7d7f310 SetMenuBarObscured + 372
4   HIToolbox                           0x00000001b7d7eee8 _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 172
5   HIToolbox                           0x00000001b7d78fcc _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 296
6   HIToolbox                           0x00000001b7d3fcd0 _NotifyEventLoopObservers + 176
7   HIToolbox                           0x00000001b7d7896c AcquireEventFromQueue + 432
8   HIToolbox                           0x00000001b7d67c84 ReceiveNextEventCommon + 320
9   HIToolbox                           0x00000001b7d67b2c _BlockUntilNextEventMatchingListInModeWithFilter + 72
10  AppKit                              0x00000001b190d84c _DPS

5.7069359281636745
5.964712055932069
5.9513652337191205
5.931550288895814
5.862426072720123
5.853048067150277
5.874203950631693
5.780830873335151


1   HIToolbox                           0x00000001b7d6790c _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 44
2   HIToolbox                           0x00000001b7da6478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x00000001b32b9214 _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x00000001b32b7e10 CGSSnarfAndDispatchDatagrams + 808
5   SkyLight                            0x00000001b35dfa9c SLSGetNextEventRecordInternal + 344
6   SkyLight                            0x00000001b33e0fb0 SLEventCreateNextEvent + 16
7   HIToolbox                           0x00000001b7d74b58 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 60
8   HIToolbox                           0x00000001b7d74ae0 _ZL14MessageHandlerP12__CFMachPortPvlS1_ + 60
9   CoreFoundation                      0x00000001ae6b9410 __CFMachPortPerform + 260
10  CoreFoundation                      0x00000001ae689f98 __CFRUNLO

5.058906540038331
5.588173223478752
5.55304657323261
5.6101771277680745
5.590652353228799
5.761061035258416


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [3]:
waktu_jatuh = catatan_waktu2[0] - catatan_waktu1[0]
print(waktu_jatuh, " detik pada render 6 FPS dan rekam 120 FPS") 

2.3891780376434326  detik pada render 6 FPS dan rekam 120 FPS


In [4]:
waktu_jatuh = waktu_jatuh/(120/fps)

In [5]:
print(Pembatas_atas, Pembatas_bawah)

754 1191


In [6]:
print(waktu_jatuh, " detik pada real time") 

0.11470167082468954  detik pada real time


In [7]:
percepatan_gravitasi = 0.3/((0.2*waktu_jatuh) + (0.5*waktu_jatuh*waktu_jatuh))
print(percepatan_gravitasi)

10.163093665060671
